In [1]:
import numpy as np
import pandas as pd
import nibabel as nib

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import matplotlib.pyplot as plt
import seaborn as sns
import os, sys, time
import itertools

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
data_dir = "/user/pfilipia/home/inria/chu_nice_inria/patients_dmri"

if not 'file_suffix' in globals() and not 'file_suffix' in vars():
    file_suffix = None
    

def get_connections_file(patient_id, file_suffix = None):

    if file_suffix is None:
        file_suffix = "_after_shift.csv"
        print("File suffix is not defined, using '%s' instead." % file_suffix)
    
    return os.path.join(
        data_dir, 
        "patient%02d/bids/sub-patient%02d/ses-presurgical/connectivity/connections_common_avg_seed5k%s" % (
            patient_id, patient_id, file_suffix
        )
    )


In [3]:
data_pd = {}
electrode_locations = {}
stimulation_site_locations = {}


In [4]:
def average_radius_in_spherical_roi(diameter):
    radius = diameter / 2
    radius_sqr = radius * radius

    total_radius = 0
    voxels = 0
    
    for z in np.arange(-radius, radius + 1):
        for y in np.arange(-radius, radius + 1):
            for x in np.arange(-radius, radius + 1):
                
                local_radius_sqr = x * x + y * y + z * z
                if local_radius_sqr <= radius_sqr:
                    total_radius += np.sqrt(local_radius_sqr)
                    voxels += 1
                    
    return total_radius / voxels
                    

roi_diameter = 10
average_radius = average_radius_in_spherical_roi(roi_diameter)

In [5]:
def get_electrode_coords(electrode_locations, electrode_id):

    if isinstance(electrode_id, str):
        electrode_id = int(electrode_id.replace('e', ''))

    electrode_coords = np.where(electrode_locations==electrode_id)

    return electrode_coords[0][0], electrode_coords[1][0]


def get_stimulation_site_coords(electrode_locations, stimulation_site_locations, stimulation_site):

    nearest_electrode_coords = get_electrode_coords(
        electrode_locations, stimulation_site_locations[stimulation_site][0]
    )
    x_stim = nearest_electrode_coords[1] - 0.50
    y_stim = nearest_electrode_coords[0] + 0.60

    shift = 0.5 # 0.75

    if 'w' in stimulation_site_locations[stimulation_site][1]:
        x_stim -= shift
    elif 'e' in stimulation_site_locations[stimulation_site][1]:
        x_stim += shift

    if 'n' in stimulation_site_locations[stimulation_site][1]:
        y_stim -= shift
    elif 's' in stimulation_site_locations[stimulation_site][1]:
        y_stim += shift

    return [x_stim, y_stim]
        

In [6]:
def show_intensities(data_pd, electrode_locations, stimulation_site_locations):

    plot_titles = ['delays', 'amplitudes', 'fibers_num', 'distances']

    stimulation_sites = data_pd.iloc[:, 0].unique()
    stimulation_sites_num = len(stimulation_sites)

    fig, ax = plt.subplots(stimulation_sites_num, 4, figsize=(15, 3 * stimulation_sites_num))

    for i in range(stimulation_sites_num):
        stimulation_site = stimulation_sites[i]

        x = []
        y = []

        s_latencies = []
        s_amplitudes = []
        s_fibers_num = []
        s_distances = []

        for stimulation_record in data_pd[data_pd.iloc[:, 0] == stimulation_site].iterrows():

            electrode_coords = get_electrode_coords(electrode_locations, stimulation_record[1][1])

            x.append(electrode_coords[1])
            y.append(electrode_coords[0])

            if stimulation_record[1][10] > 0:
                s_latencies.append(stimulation_record[1][10])
                s_amplitudes.append(abs(stimulation_record[1][11]))
            else:
                s_latencies.append(0)
                s_amplitudes.append(0)

            s_fibers_num.append(stimulation_record[1][3])
            s_distances.append(stimulation_record[1][5])

        s = np.array([s_latencies, s_amplitudes, s_fibers_num, s_distances])
        stim_coords = get_stimulation_site_coords(
            electrode_locations, stimulation_site_locations, stimulation_site
        )
    
        for j in range(4):

            s_norm = s[j] / np.sum(np.array(s[j]))
#             s_norm = s[j] / np.sqrt((np.sum(np.array(s[j])**2)))

            ax[i, j].scatter(x, y, s=1, color='black')
            ax[i, j].scatter(x, y, s=1000*s_norm)
            ax[i, j].text(stim_coords[0], stim_coords[1], '*', fontsize=24, color='magenta')

            plt.sca(ax[i, j])
            plt.xticks([])
            plt.yticks([])
            plt.xlim([-1, electrode_locations.shape[1]])
            plt.ylim([-1, electrode_locations.shape[0]])

            if j == 0:
                plt.ylabel(stimulation_site)

            plt.gca().invert_yaxis()
            plt.title(plot_titles[j])


In [7]:
def binary_value(val):
    result = 0
    if val > 0:
        result = 1
    return result

def show_connections(data_pd, electrode_locations):

    stimulation_sites = data_pd.iloc[:, 0].unique()
    stimulation_sites_num = len(stimulation_sites)

    count_intersect = 0
    count_both = 0
    
    count_dmri_only = 0
    count_des_only = 0
    
    fig, ax = plt.subplots(stimulation_sites_num, 1, figsize=(5, 4 * stimulation_sites_num))

    for i in range(stimulation_sites_num):
        stimulation_site = stimulation_sites[i]

        x = []
        y = []

        s_latencies = []
        s_fibers_num = []

        for stimulation_record in data_pd[data_pd.iloc[:, 0] == stimulation_site].iterrows():

            electrode_coords = get_electrode_coords(electrode_locations, stimulation_record[1][1])

            x.append(electrode_coords[1])
            y.append(electrode_coords[0])

            s_latencies.append(binary_value(stimulation_record[1][10]))
            s_fibers_num.append(binary_value(stimulation_record[1][3]))

        s_fibers_num = np.array(s_fibers_num)
        s_latencies = np.array(s_latencies)
        
        count_intersect = count_intersect + np.sum(s_fibers_num == s_latencies)
        count_both      = count_both + np.sum(s_fibers_num + s_latencies > 1)

        count_dmri_only = count_dmri_only + np.sum(s_fibers_num > s_latencies)
        count_des_only  = count_des_only + np.sum(s_latencies > s_fibers_num)

        ax[i].scatter(x, y, s=1, color='black')
        ax[i].scatter(x, y, s=100*s_fibers_num, color='black', label='dMRI')
        ax[i].scatter(x, y, s=200*s_latencies, marker='x', color='red', label='DES')

        plt.sca(ax[i])
        plt.xticks([])
        plt.yticks([])
        plt.xlim([-1, electrode_locations.shape[1]])
        plt.ylim([-1, electrode_locations.shape[0]])

        plt.ylabel(stimulation_site)

        plt.gca().invert_yaxis()
        plt.title("connections dMRI vs. DES")
        plt.legend()

    total_count = count_intersect + count_dmri_only + count_des_only
    print("dMRI = DES: %3d (%3d%%)" % (count_intersect, 100 * count_intersect / total_count))
    print("dMRI & DES: %3d (%3d%%)\n" % (count_both, 100 * count_both / total_count))
    print("dMRI only : %3d (%3d%%)" % (count_dmri_only, 100 * count_dmri_only / total_count))
    print("DES only  : %3d (%3d%%)" % (count_des_only, 100 * count_des_only / total_count))
        

In [8]:
def rotate_electrode_locations(electrode_locations):
    
    return np.flipud(np.fliplr(electrode_locations))


def rotate_stimulation_site_locations(stimulation_site_locations):
    
    direction_mapping = {
        'n': 's', 'e': 'w', 's': 'n', 'w': 'e',
        'ne': 'sw', 'nw': 'se', 'se': 'nw', 'sw': 'ne'
    }
    
    for stimulation_site in stimulation_site_locations:
        
        stimulation_site_locations[stimulation_site][1] = direction_mapping[stimulation_site_locations[stimulation_site][1]]
        
    return stimulation_site_locations
    
    

# Patient #1

In [30]:
patient_id = 1 

electrode_locations[patient_id] = rotate_electrode_locations(np.array([
    [ 1,  5,  0,  0,  0,  0,  0,  0],
    [ 2,  6,  0,  0,  0,  0,  0,  0],
    [ 3,  7,  0,  0,  0,  0,  0,  0],
    [ 4,  8,  0,  0,  0,  0,  0,  0],
    [ 0,  0, 14, 13, 12, 11, 10,  9]
]))

stimulation_site_locations[patient_id] = rotate_stimulation_site_locations({
    's00': [  1,  'w' ],
    's01': [  2, 'sw' ],
    's02': [  4,  'w' ],
    's03': [  6,  'e' ],
    's04': [  7,  'e' ],
    's05': [  8, 'se' ],
    's06': [  5, 'ne' ],
    's07': [ 13,  'n' ],
    's08': [ 12,  'n' ],
    's09': [  1,  'n' ]
})

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


# Patient #3

In [31]:
patient_id = 3

electrode_locations[patient_id] = np.array([
    [ 1,  0,  0,  0,  0,  0 ],
    [ 2,  0,  0,  0,  0,  0 ],
    [ 3,  0,  0,  0,  0,  0 ],
    [ 4,  0,  0,  0,  0,  0 ],
    [ 0,  0,  0,  0,  0,  0 ],
    [ 5,  6,  7,  8,  9, 10 ]
])

stimulation_site_locations[patient_id] = {
    's00': [  1,  'e' ],
    's01': [  2,  'e' ],
    's02': [  3,  'e' ],
    's03': [  4, 'se' ],
    's04': [  6,  's' ],
    's05': [  7,  's' ],
    's06': [  8,  's' ],
    's07': [  9,  's' ],
    's08': [ 10,  's' ],
    's09': [  9,  'n' ]
}

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))
    

File suffix is not defined, using '_after_shift.csv' instead.


# Patient #4

In [32]:
patient_id = 4

electrode_locations[patient_id] = np.array([
    [  0,  0,  0,  0,  0,  9,  0,  0 ],
    [  0,  0,  0,  0, 10,  0,  0,  0 ],
    [  0,  0,  0, 11,  0,  0,  0,  0 ],
    [  0,  0, 12,  0,  0,  0,  0,  0 ],
    [  0, 13,  0,  0,  0,  0,  0,  0 ],
    [ 14,  0,  0,  0,  4,  3,  2,  1 ],
    [  0,  0,  0,  0,  8,  7,  6,  5 ]
])

stimulation_site_locations[patient_id] = {
    's00': [  9, 'se' ],
    's01': [ 10, 'se' ],
    's02': [ 11, 'se' ],
    's03': [ 12, 'se' ],
    's04': [ 13, 'se' ],
    's05': [ 14, 'se' ],
    's06': [  1,  'n' ],
    's07': [  2,  'n' ],
    's08': [  3,  'n' ],
    's09': [  4,  'n' ],
    's10': [  5,  's' ],
    's11': [  6,  's' ],
    's12': [  7, 'sw' ],
    's13': [  8, 'nw' ]    
}

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


# Patient #6

In [33]:
patient_id = 6

electrode_locations[patient_id] = np.array([
    [  0,  0,  0,  8,  4,  0 ],
    [  0,  0,  0,  7,  3,  0 ],
    [  0,  0,  0,  6,  2,  0 ],
    [  0,  0,  0,  5,  1,  0 ],
    [ 14, 13, 12, 11, 10,  9 ]
])

stimulation_site_locations[patient_id] = {
    's01': [  5,  'w' ],
    's02': [ 13,  'n' ],
    's03': [ 14, 'nw' ],
    's04': [  2,  'e' ],
    's05': [  3,  'e' ],
    's06': [  4,  'e' ],
    's07': [  8,  'w' ],
    's08': [  7,  'w' ],
    's09': [  6,  'w' ]
}

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


# Patient #7

In [34]:
patient_id = 7

# electrode_locations[patient_id] = np.array([
#     [  9, 10, 11,  0,  0,  0 ],
#     [  0,  0,  0, 12, 13, 14 ],
#     [  0,  0,  0,  0,  0,  0 ],
#     [  0,  5,  6,  7,  8,  0 ],
#     [  0,  1,  2,  3,  4,  0 ]
# ])

electrode_locations[patient_id] = np.array([
    [  9, 10, 11, 12, 13, 14 ],
    [  0,  0,  0,  0,  0,  0 ],
    [  0,  5,  6,  7,  8,  0 ],
    [  0,  1,  2,  3,  4,  0 ]
])

stimulation_site_locations[patient_id] = {
    's00': [ 14,  's' ],
    's01': [  1, 'se' ],
    's02': [  2,  's' ],
    's03': [  3,  's' ],
    's04': [  4,  's' ],
    's05': [  5, 'ne' ],
    's06': [  6,  'n' ],
    's07': [  7,  'n' ],
    's08': [  8,  'n' ],
    's09': [ 12,  's' ],
    's10': [ 13,  's' ]
}

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


# Patient #8

In [35]:
patient_id = 8

electrode_locations[patient_id] = np.array([
    [  0,  0,  0,  0,  0,  0 ],
    [  0,  4,  3,  2,  1,  0 ],
    [  0,  8,  7,  6,  5,  0 ],
    [  0,  0,  0,  0,  0,  0 ]
])

stimulation_site_locations[patient_id] = {
    's00': [  1,  'n' ],
    's01': [  2,  'n' ],
    's02': [  3,  'n' ],
    's03': [  4,  'n' ],
    's04': [  5,  's' ],
    's05': [  6,  's' ],
    's06': [  7,  's' ],
    's07': [  8,  's' ],
}

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


# Patient #10

In [36]:
patient_id = 10

electrode_locations[patient_id] = rotate_electrode_locations(np.array([
    [ 8,  4,  0,  0,  0,  0,  0,  0],
    [ 7,  3,  0,  0,  0,  0,  0,  0],
    [ 6,  2,  0,  0,  0,  0,  0,  0],
    [ 5,  1,  0,  0,  0,  0,  0,  0],
    [ 0,  0, 14, 13, 12, 11, 10,  9]
]))

stimulation_site_locations[patient_id] = rotate_stimulation_site_locations({
    's01': [  9,  's' ],
    's02': [ 10,  's' ],
    's03': [ 11,  's' ],
    's04': [ 12,  'n' ],
    's05': [ 13,  's' ],
    's06': [ 14,  's' ],
    's07': [  1,  'e' ],
    's08': [  2,  'e' ],
    's09': [  3,  'e' ],
    's10': [  4,  'e' ]
})

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


# Patient #11

In [37]:
patient_id = 11

electrode_locations[patient_id] = rotate_electrode_locations(np.array([
    [ 8,  4,  0,  0,  0,  0,  0],
    [ 7,  3,  0,  0,  0,  0,  0],
    [ 6,  2,  0,  0,  0,  0,  0],
    [ 5,  1,  0,  0,  0,  0,  0],
    [ 0, 14, 13, 12, 11, 10,  9]
]))

stimulation_site_locations[patient_id] = rotate_stimulation_site_locations({
    's01': [  9,  'n' ],
    's02': [ 10,  'n' ],
    's03': [ 11,  's' ],
    's04': [ 12,  's' ],
    's05': [ 13,  's' ],
    's06': [ 14,  's' ],
    's07': [  1, 'ne' ],
    's08': [  2,  'e' ],
    's09': [  3,  'e' ],
    's10': [  4,  'e' ],
    's11': [  8,  'w' ],
    's12': [  6,  'w' ]
})

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


# Patient #12

In [38]:
patient_id = 12

electrode_locations[patient_id] = rotate_electrode_locations(np.array([
    [ 0,  9,  0,  0,  0],
    [ 0, 10,  5,  1,  0],
    [ 0, 11,  6,  2,  0],
    [ 0, 12,  7,  3,  0],
    [ 0, 13,  8,  4,  0],
    [ 0, 14,  0,  0,  0]
]))

stimulation_site_locations[patient_id] = rotate_stimulation_site_locations({
    's01': [  1,  'e' ],
    's02': [  2,  'e' ],
    's03': [  3,  'e' ],
    's04': [  4,  'e' ],
    's05': [  5,  'e' ],
    's06': [  6,  'e' ],
    's07': [  7,  'e' ],
    's08': [  8,  'e' ],
    's09': [ 10,  'e' ],
    's10': [ 11,  'e' ],
    's11': [ 12,  'e' ],
    's12': [ 13,  'e' ]
})

try:
    data_pd[patient_id] = pd.read_csv(get_connections_file(patient_id, file_suffix), skipinitialspace=True)
except:
    print("Couldn't load Patient #%d data using file suffix: %s" % (patient_id, file_suffix))


File suffix is not defined, using '_after_shift.csv' instead.


In [18]:
patient_labels = {
     1 : 1,
     3 : 2,
     4 : 3,
     6 : 4,
     7 : 5,
     8 : 6,
    10 : 7,
    11 : 8,
    12 : 9
}

In [19]:
def fetch_connections_data(data_pd, column_id):
    
    stimulation_sites = sorted(data_pd.iloc[:, 0].unique())
    stimulation_sites_num = len(stimulation_sites)

    electrodes = sorted(data_pd.iloc[:, 1].unique())
    electrodes_num = len(electrodes)

    result_matrix = np.zeros([stimulation_sites_num, electrodes_num])
    
    for stimulation_record in data_pd.iterrows():
        try:
            row_id = stimulation_sites.index(stimulation_record[1][0])
            col_id = electrodes.index(stimulation_record[1][1])
            value = stimulation_record[1][column_id]
            if not np.isnan(value):
                result_matrix[row_id, col_id] = value
        except:
            None
            
    return result_matrix


In [20]:
def get_streamlines_count_matrix(data_pd):

    return fetch_connections_data(data_pd, 3)


def get_streamlines_log_count_matrix(data_pd):

    return np.log10(1 + get_streamlines_count_matrix(data_pd))


def get_streamlines_min_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 5)


def get_streamlines_sqrt_min_length_matrix(data_pd):

    return np.sqrt(2 * average_radius + fetch_connections_data(data_pd, 5))


def get_streamlines_log_min_length_matrix(data_pd):
    
    return np.log10(1 + get_streamlines_min_length_matrix(data_pd))


def get_streamlines_max_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 6)


def get_streamlines_avg_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 7)


def get_streamlines_median_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 8)


def get_streamlines_std_length_matrix(data_pd):

    return fetch_connections_data(data_pd, 9)


def get_evoked_potential_delay_matrix(data_pd):
    
    return fetch_connections_data(data_pd, 10)


def get_evoked_potential_amplitude_matrix(data_pd):
    
    return np.abs(fetch_connections_data(data_pd, 11))


def get_smoothwm_surface_distance_matrix(data_pd):

    return fetch_connections_data(data_pd, 14)


def get_pial_surface_distance_matrix(data_pd):

    return fetch_connections_data(data_pd, 15)


def get_trk_streamlines_count_matrix(data_pd):

    return fetch_connections_data(data_pd, 16)


def get_trk_streamlines_min_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 17)


def get_trk_streamlines_10_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 18)


def get_trk_streamlines_20_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 19)


def get_trk_streamlines_30_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 20)


def get_trk_streamlines_40_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 21)


def get_trk_streamlines_median_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 22)


def get_trk_streamlines_max_length_matrix(data_pd):

    return 2 * average_radius + fetch_connections_data(data_pd, 23)


def get_mean_fa_matrix(data_pd):

    return fetch_connections_data(data_pd, 24)


def get_mean_md_matrix(data_pd):

    return fetch_connections_data(data_pd, 26)


def get_mean_ad_matrix(data_pd):

    return fetch_connections_data(data_pd, 28)


def get_mean_rd_matrix(data_pd):

    return fetch_connections_data(data_pd, 30)


def get_mean_rtop_matrix(data_pd):

    return np.power(np.maximum(0, fetch_connections_data(data_pd, 32)), 1.0/3)


def get_mean_rtap_matrix(data_pd):

    return np.power(np.maximum(0, fetch_connections_data(data_pd, 33)), 1.0/2)


def get_mean_rtpp_matrix(data_pd):

    return np.maximum(0, fetch_connections_data(data_pd, 34))


def get_mean_msd_matrix(data_pd):

    return np.maximum(0, fetch_connections_data(data_pd, 35))


def get_mean_qiv_matrix(data_pd):

    return np.maximum(0, fetch_connections_data(data_pd, 36))


def get_mean_ng_matrix(data_pd):

    return np.maximum(0, fetch_connections_data(data_pd, 37))


def get_mean_ng_perp_matrix(data_pd):

    return np.maximum(0, fetch_connections_data(data_pd, 38))


def get_mean_ng_par_matrix(data_pd):

    return np.maximum(0, fetch_connections_data(data_pd, 39))


In [21]:
def get_inversed_streamlines_count_matrix(data_pd):

    return 1.0 / (1.0 + get_streamlines_count_matrix(data_pd))


def get_inversed_streamlines_log_count_matrix(data_pd):

    return 1.0 / (1.0 + get_streamlines_log_count_matrix(data_pd))


def get_inversed_length_based_matrix(data_pd, callback_fun):

    return 1.0 / (1.0 + callback_fun(data_pd))


def get_inversed_streamlines_min_length_matrix(data_pd):

    return get_inversed_length_based_matrix(data_pd, get_streamlines_min_length_matrix)


def get_inversed_streamlines_max_length_matrix(data_pd):

    return get_inversed_length_based_matrix(data_pd, get_streamlines_max_length_matrix)


def get_inversed_streamlines_avg_length_matrix(data_pd):

    return get_inversed_length_based_matrix(data_pd, get_streamlines_avg_length_matrix)


def get_inversed_streamlines_median_length_matrix(data_pd):

    return get_inversed_length_based_matrix(data_pd, get_streamlines_median_length_matrix)


def get_inversed_streamlines_std_length_matrix(data_pd):

    return get_inversed_length_based_matrix(data_pd, get_streamlines_std_length_matrix)


def get_inversed_smoothwm_surface_distance_matrix(data_pd):

    return get_inversed_length_based_matrix(data_pd, get_smoothwm_surface_distance_matrix)


def get_inversed_pial_surface_distance_matrix(data_pd):

    return get_inversed_length_based_matrix(data_pd, get_pial_surface_distance_matrix)


In [22]:
def get_count_based_connectivity_matrix(data_pd, mask = None):
       
    result_matrix = get_streamlines_count_matrix(data_pd)
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    return result_matrix / np.sum(result_matrix)

def get_log_count_based_connectivity_matrix(data_pd, mask = None):
       
    result_matrix = get_streamlines_log_count_matrix(data_pd)
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    return result_matrix / np.sum(result_matrix)


In [23]:
def get_fractionally_scalled_connectivity_matrix(data_pd, mask = None):

    input_matrix = get_streamlines_count_matrix(data_pd)
    result_matrix = np.zeros_like(input_matrix)

    for row_id in range(input_matrix.shape[0]):
        for col_id in range(input_matrix.shape[1]):
            scaling_factor = np.sum(input_matrix[row_id, :col_id])
            scaling_factor += np.sum(input_matrix[row_id, col_id+1:])
            scaling_factor += np.sum(input_matrix[:row_id, col_id])
            scaling_factor += np.sum(input_matrix[row_id+1:, col_id])
            result_matrix[row_id, col_id] = input_matrix[row_id, col_id] / scaling_factor
    
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    
    return result_matrix / np.sum(result_matrix)
    

In [24]:
def get_length_based_connectivity_matrix(data_pd, callback_fun, mask = None, distance_threshold = None):
       
    result_matrix = callback_fun(data_pd)
    
    if distance_threshold:
        result_matrix[result_matrix > distance_threshold] = 0
    
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    
    return result_matrix / np.sum(result_matrix)
    

def get_streamlines_min_length_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_length_based_connectivity_matrix(
        data_pd, get_streamlines_min_length_matrix, mask, distance_threshold
    )


def get_streamlines_max_length_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_length_based_connectivity_matrix(
        data_pd, get_streamlines_max_length_matrix, mask, distance_threshold
    )


def get_streamlines_avg_length_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_length_based_connectivity_matrix(
        data_pd, get_streamlines_avg_length_matrix, mask, distance_threshold
    )


def get_streamlines_std_length_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_length_based_connectivity_matrix(
        data_pd, get_streamlines_std_length_matrix, mask, distance_threshold
    )


def get_smoothwm_surface_distance_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_length_based_connectivity_matrix(
        data_pd, get_smoothwm_surface_distance_matrix, mask, distance_threshold
    )


def get_pial_surface_distance_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_length_based_connectivity_matrix(
        data_pd, get_pial_surface_distance_matrix, mask, distance_threshold
    )
        

In [25]:
def get_inversed_length_based_connectivity_matrix(data_pd, callback_fun, mask = None):
    
    result_matrix = 1 / (1 + callback_fun(data_pd))

    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    
    return result_matrix / np.sum(result_matrix)


def get_inversed_streamlines_min_length_based_connectivity_matrix(data_pd, mask = None):

    return get_inversed_length_based_connectivity_matrix(
        data_pd, get_streamlines_min_length_matrix, mask
    )


def get_inversed_streamlines_max_length_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_inversed_length_based_connectivity_matrix(
        data_pd, get_streamlines_max_length_matrix, mask
    )


def get_inversed_streamlines_avg_length_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_inversed_length_based_connectivity_matrix(
        data_pd, get_streamlines_avg_length_matrix, mask
    )


def get_inversed_streamlines_std_length_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_inversed_length_based_connectivity_matrix(
        data_pd, get_streamlines_std_length_matrix, mask
    )


def get_inversed_smoothwm_surface_distance_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_inversed_length_based_connectivity_matrix(
        data_pd, get_smoothwm_surface_distance_matrix, mask
    )


def get_inversed_pial_surface_distance_based_connectivity_matrix(data_pd, mask = None, distance_threshold = None):

    return get_inversed_length_based_connectivity_matrix(
        data_pd, get_pial_surface_distance_matrix, mask
    )


In [26]:
def get_delay_based_connectivity_matrix(data_pd, mask = None):
    
    result_matrix = get_evoked_potential_delay_matrix(data_pd)
    result_matrix[result_matrix > 0] -= np.min(result_matrix[result_matrix > 0]) - 1
    
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    
    return result_matrix / np.sum(result_matrix)


def get_inversed_delay_based_connectivity_matrix(data_pd, mask = None):
    
    result_matrix = 1 / (1 + get_evoked_potential_delay_matrix(data_pd))
    
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    
    return result_matrix / np.sum(result_matrix)


In [27]:
def get_amplitude_based_connectivity_matrix(data_pd, mask = None):
    
    result_matrix = get_evoked_potential_amplitude_matrix(data_pd)
    
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    
    return result_matrix / np.sum(result_matrix)


def get_inversed_amplitude_based_connectivity_matrix(data_pd, mask = None):
    
    result_matrix = 1 / (1 + get_evoked_potential_amplitude_matrix(data_pd))
    
    if not mask is None:
        result_matrix[np.logical_not(mask)] = 0
    
    return result_matrix / np.sum(result_matrix)


In [28]:
def get_intraelectrode_distance_matrix(electrode_locations, stimulation_site_locations):
    
    stimulation_sites = sorted(stimulation_site_locations)
    stimulation_sites_num = len(stimulation_sites)

    electrodes = sorted(electrode_locations[electrode_locations > 0])
    electrodes_num = len(electrodes)

    result_matrix = np.zeros([stimulation_sites_num, electrodes_num])
    
    for stimulation_site_id in stimulation_sites:
        row_id = stimulation_sites.index(stimulation_site_id)

        for electrode_id in electrodes:
            col_id = electrodes.index(electrode_id)
            
            stimulation_site_coords = np.array(get_electrode_coords(
                electrode_locations, stimulation_site_locations[stimulation_site_id][0]
            ), dtype=float)
            
            shift = 0.5

            if 'w' in stimulation_site_locations[stimulation_site_id][1]:
                stimulation_site_coords[0] -= shift
            elif 'e' in stimulation_site_locations[stimulation_site_id][1]:
                stimulation_site_coords[0] += shift

            if 'n' in stimulation_site_locations[stimulation_site_id][1]:
                stimulation_site_coords[1] -= shift
            elif 's' in stimulation_site_locations[stimulation_site_id][1]:
                stimulation_site_coords[1] += shift

            electrode_coords = np.array(get_electrode_coords(electrode_locations, electrode_id))
            result_matrix[row_id, col_id] = np.sqrt(np.sum((stimulation_site_coords - electrode_coords) ** 2))
            
    return result_matrix
    